In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from time import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from neuralforecast.utils import AirPassengersDF

from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast import NeuralForecast
from neuralforecast.auto import NHITS

import matplotlib.pyplot as plt

from IPython.display import clear_output
from copy import deepcopy

import logging
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)

/home/patel_zeel/neuralforecast/neuralforecast/utils.py:252: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),
/home/patel_zeel/neuralforecast/neuralforecast/utils.py:264: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M").values,
/home/patel_zeel/neuralforecast/neuralforecast/utils.py:265: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M").values,
/home/patel_zeel/neuralforecast/neuralforecast/utils.py:281: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because 

In [2]:
df = pd.read_csv('https://datasets-nixtla.s3.amazonaws.com/EPF_FR_BE.csv')
df['ds'] = pd.to_datetime(df['ds'])
df.head(), df.shape

(  unique_id                  ds      y  gen_forecast  system_load  week_day
 0        FR 2015-01-01 00:00:00  53.48       76905.0      74812.0         3
 1        FR 2015-01-01 01:00:00  51.93       75492.0      71469.0         3
 2        FR 2015-01-01 02:00:00  48.76       74394.0      69642.0         3
 3        FR 2015-01-01 03:00:00  42.27       72639.0      66704.0         3
 4        FR 2015-01-01 04:00:00  38.41       69347.0      65051.0         3,
 (32160, 6))

In [3]:
static_df = pd.read_csv('https://datasets-nixtla.s3.amazonaws.com/EPF_FR_BE_static.csv')
static_df.head(), static_df.shape

(  unique_id  market_0  market_1
 0        FR         1         0
 1        BR         0         1,
 (2, 3))

In [4]:
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000

def fit(compile=False):
    horizon = 24 # day-ahead daily forecast
    models = [NHITS(h = horizon,
                    input_size = 5*horizon,
                    futr_exog_list = ['gen_forecast', 'week_day'], # <- Future exogenous variables
                    hist_exog_list = ['system_load'], # <- Historical exogenous variables
                    stat_exog_list = ['market_0', 'market_1'], # <- Static exogenous variables
                    scaler_type = 'robust', max_steps=100)]
    if compile:
        models = [torch.compile(model) for model in models]
    
    nf = NeuralForecast(models=models, freq='H')
    nf.fit(df=df,
       static_df=static_df)

In [5]:
eager_times = []
compiled_times = []
for i in range(5):
    _, elapsed = timed(lambda: fit(compile=False))
    print(f'Elapsed time: {elapsed:.2f} seconds')
    eager_times.append(elapsed)

for i in range(5):
    _, elapsed = timed(lambda: fit(compile=True))
    print(f'Elapsed time: {elapsed:.2f} seconds')
    compiled_times.append(elapsed)
    
median_eager = np.median(eager_times)
median_compiled = np.median(compiled_times)
speedup = median_eager / median_compiled
print(f'Median eager time: {median_eager:.2f} seconds')
print(f'Median compiled time: {median_compiled:.2f} seconds')
print(f'Speedup: {speedup:.2f}x')

Seed set to 1


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Seed set to 1


...

Elapsed time: 3.01 seconds


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Elapsed time: 2.75 seconds
Median eager time: 2.92 seconds
Median compiled time: 2.97 seconds
Speedup: 0.98x
